# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_cities = pd.read_csv("clean_cities.csv")
vacation_cities.head()

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Wind,Cloudiness,Date
0,0,iqaluit,63.75,-68.51,55.20,76,2.24,75,1595892169
1,1,bilibino,68.05,166.44,52.45,54,5.50,94,1595892215
2,2,saint-philippe,-21.36,55.77,61.39,82,4.70,94,1595892111
3,3,olafsvik,64.89,-23.71,48.36,74,10.65,32,1595891954
4,4,caiaponia,-16.96,-51.81,68.54,40,2.48,0,1595892216


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
humidity = vacation_cities['Humidity']
locations = vacation_cities[['Lat', 'Lon']]
maxhum = humidity.max()

In [19]:
fig1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhum, point_radius=2)
fig1.add_layer(heat_layer)
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_cities_weather_df = vacation_cities.loc[(vacation_cities['Temp']> 70) & (vacation_cities['Temp']<= 80) & (vacation_cities['Wind']< 10) & (vacation_cities['Cloudiness'] == 0)]
vacation_cities_weather_df = vacation_cities_weather_df.reset_index(drop=True)
vacation_cities_weather_df = vacation_cities_weather_df.drop(columns = "Unnamed: 0")
vacation_cities_weather_df

,City,Lat,Lon,Temp,Humidity,Wind,Cloudiness,Date
0,tripoli,32.88,13.19,77.67,72,8.43,0,1595892187
1,riorges,46.04,4.05,74.97,53,3.36,0,1595892222
2,binzhou,37.37,118.02,75.72,79,3.58,0,1595892230
3,mizdah,31.45,12.98,71.82,58,9.01,0,1595892230
4,montagnana,45.23,11.46,72.82,84,2.82,0,1595892250
5,hun,29.13,15.95,74.03,64,3.67,0,1595892256
6,guarapari,-20.67,-40.50,73.90,78,4.70,0,1595892258
7,kasongo-lunda,-6.47,16.82,70.36,33,2.21,0,1595892270
8,nizhniy kuranakh,58.84,125.49,74.17,75,6.11,0,1595892277
9,lagoa,39.05,-27.98,73.80,88,4.63,0,1595892290


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#add hotels column to DF
vacation_cities_weather_df["Hotel Name"]= ""
hotel_df = vacation_cities_weather_df


In [7]:
#build url
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius": 5000,
         "types": "hotel",
         "keyword": "hotel",
         "key": g_key}

In [8]:
# Use the lat/lng recovered to identify hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    cities = row["City"]
    params["location"] = (f"{lat},{lng}")
    
    #print(f"Results for Index {index} : {cities}")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {cities} is {results[0]['name']}.")
        print(f"-----------------------")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError , IndexError):
        
        print(f"No hotel in {cities} close by.")
        print(f"-------")

Closest hotel in tripoli is Radisson Blu Al Mahary Hotel.
-----------------------
Closest hotel in riorges is Château de Champlong.
-----------------------
Closest hotel in binzhou is Binzhou Hotel.
-----------------------
Closest hotel in mizdah is فندق مزدة الكبير.
-----------------------
Closest hotel in montagnana is Castello Bevilacqua.
-----------------------
Closest hotel in hun is Alrwasi Hotel.
-----------------------
Closest hotel in guarapari is Bristol Guarapari Residence Hotel.
-----------------------
No hotel in kasongo-lunda close by.
-------
Closest hotel in nizhniy kuranakh is Gostinitsa "Zolotinka".
-----------------------
Closest hotel in lagoa is Graciosa Resort - Biosphere Island Hotel.
-----------------------
Closest hotel in altamira is Xingu Praia Hotel.
-----------------------
Closest hotel in vila velha is Hotel Santorini.
-----------------------
Closest hotel in alcudia is Iberostar Selection Playa de Muro Village.
-----------------------
Closest hotel in sub

In [9]:
hotel_df

,City,Lat,Lon,Temp,Humidity,Wind,Cloudiness,Date,Hotel Name
0,tripoli,32.88,13.19,77.67,72,8.43,0,1595892187,Radisson Blu Al Mahary Hotel
1,riorges,46.04,4.05,74.97,53,3.36,0,1595892222,Château de Champlong
2,binzhou,37.37,118.02,75.72,79,3.58,0,1595892230,Binzhou Hotel
3,mizdah,31.45,12.98,71.82,58,9.01,0,1595892230,فندق مزدة الكبير
4,montagnana,45.23,11.46,72.82,84,2.82,0,1595892250,Castello Bevilacqua
5,hun,29.13,15.95,74.03,64,3.67,0,1595892256,Alrwasi Hotel
6,guarapari,-20.67,-40.50,73.90,78,4.70,0,1595892258,Bristol Guarapari Residence Hotel
7,kasongo-lunda,-6.47,16.82,70.36,33,2.21,0,1595892270,
8,nizhniy kuranakh,58.84,125.49,74.17,75,6.11,0,1595892277,"Gostinitsa ""Zolotinka"""
9,lagoa,39.05,-27.98,73.80,88,4.63,0,1595892290,Graciosa Resort - Biosphere Island Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
#<dt>Country</dt><dd>{Country}</dd>
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [16]:
# Add marker layer ontop of heat map
markers_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig1.add_layer(markers_layer)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))